<a href="https://colab.research.google.com/github/tarakantaacharya/TTS_ODIA/blob/main/Preprocessing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets
! pip install librosa

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import json
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
import librosa

# Function to preprocess the audio (example: resample and normalize)
def preprocess_audio(audio_data, sampling_rate, target_sr=16000):
    # Resample the audio to a target sample rate (if needed)
    if sampling_rate != target_sr:
        audio_data = librosa.resample(audio_data, orig_sr=sampling_rate, target_sr=target_sr)  # Corrected
        sampling_rate = target_sr  # Update the sampling rate to target

    # Normalize the audio (scaling the waveform between -1 and 1)
    audio_data = librosa.util.normalize(audio_data)

    return audio_data, sampling_rate

# Function to preprocess the text (tokenize)
def preprocess_text(text, tokenizer):
    # Tokenize the text into input IDs
    tokens = tokenizer(text, return_tensors="pt")
    return tokens['input_ids'].squeeze(0).tolist()  # Get tokenized text as list of IDs

# Function to preprocess data from the dataset
def preprocess_data_from_dataset(dataset, index, tokenizer):
    # Extract the audio data and sampling rate
    audio_data = dataset[index]['audio']['array']
    sampling_rate = dataset[index]['audio']['sampling_rate']

    # Preprocess the audio
    norm_audio_data, processed_sr = preprocess_audio(audio_data, sampling_rate)

    # Preprocess the text
    text = dataset[index]['text']
    tokenized_text = preprocess_text(text, tokenizer)

    # Return the preprocessed audio and tokenized text
    return norm_audio_data.tolist(), tokenized_text

# Load the dataset
dataset = load_dataset("SPRINGLab/IndicTTS_Odia")

# Load the tokenizer for the language model
tokenizer = tokenizer

processed_data = []

batch_size = 50  # Load 50 rows at a time

# Iterate over dataset in batches of 50
for i in range(0, len(dataset['train'])//11, batch_size):
    batch = dataset['train'].select(range(i, min(i + batch_size, len(dataset['train']))))  # Select batch

    for example in batch:
        norm_audio_data, tokenized_text = preprocess_data_from_dataset(batch, 0, tokenizer)

        processed_data.append({
            "audio": norm_audio_data,
            "text": tokenized_text
        })

    print(f"Processed {min(i + batch_size, len(dataset['train']))} rows out of {len(dataset['train'])}")

# Store the processed data as a JSON file
with open("processed_data.json", "w") as json_file:
    json.dump(processed_data, json_file)

print("Data has been saved to processed_data.json.")

Processed 50 rows out of 11564
Processed 100 rows out of 11564
Processed 150 rows out of 11564
Processed 200 rows out of 11564
Processed 250 rows out of 11564
Processed 300 rows out of 11564
Processed 350 rows out of 11564
Processed 400 rows out of 11564
Processed 450 rows out of 11564
Processed 500 rows out of 11564
Processed 550 rows out of 11564
Processed 600 rows out of 11564
Processed 650 rows out of 11564
Processed 700 rows out of 11564
Processed 750 rows out of 11564
Processed 800 rows out of 11564
Processed 850 rows out of 11564
Processed 900 rows out of 11564
Processed 950 rows out of 11564
Processed 1000 rows out of 11564
Processed 1050 rows out of 11564
Processed 1100 rows out of 11564
Data has been saved to processed_data.json.


In [ ]:
import pandas as pd
df = pd.read_json('processed_data.json')
len(df)

1100

In [ ]:
import numpy as np
np.array(df.iloc[0]['audio']).shape

(47226,)

In [ ]:
import numpy as np

# Example array
array_data = np.array(df.iloc[0]['audio'])

# Count the number of zeros
num_zeros = np.count_nonzero(array_data == 0)

# Count the number of non-zero elements
num_non_zeros = array_data.size - num_zeros  # Total size - zeros

print(f"Number of zeros: {num_zeros}")
print(f"Number of non-zero elements: {num_non_zeros}")

Number of zeros: 9119
Number of non-zero elements: 38107


In [ ]:
df.columns

Index(['audio', 'text'], dtype='object')

In [ ]:
print(type(df['audio']))

<class 'pandas.core.series.Series'>


In [ ]:
print(type(df['text']))

<class 'pandas.core.series.Series'>


In [ ]:
print(df.iloc[0])

audio    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
text     [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...
Name: 0, dtype: object


In [ ]:
print(df.head())

                                               audio  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                text  
0  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...  
1  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...  
2  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...  
3  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...  
4  [2, 41, 62688, 1, 41, 1, 62827, 62008, 1, 6267...  
